## Running Stan as MPI Job
This notebook will submit Stan as an MPI Job. It's running the [`mpi_test.py`](../code/mpi_test.py) script. This script takes a pre-compiled Stan model and runs it as MPI. The [`compile_model.py`](../code/compile_model.py) script will compile the model.

This example is based on the [Stan MPI Threading repo by Guido Biele](https://github.com/gbiele/Stan_MPI_Threading).

In [1]:
from azureml.core import Workspace, Environment, ComputeTarget, Experiment, Dataset, Run
from azureml.train.estimator import Estimator, Mpi
from uuid import uuid4

In [67]:
# Instantiate workspace and experiment objects
ws = Workspace.from_config()
stan_exp = Experiment(ws, 'stan-mpi-test')

# Retrieve the Environment, ComputeTarget and Dataset objects.
env = Environment.get(ws, 'stan-intelmpi')
cpu_cluster = ComputeTarget(ws, 'cpu-cluster')

# This dataset object is an AzureML FileDataset - which is pointing to the RDumps files 
# (generated from bbdata.R in the Stan_MPI_Threading repo) stored on Azure Blob Storage
dataset = Dataset.get_by_name(ws, 'rdumps')

datastore = ws.get_default_datastore()

In [68]:
def submit_experiment(nodes, processes_per_node, samples=20000, test_id=None):
    if not samples in [1000,5000,10000,20000,300000]:
        raise AttributeError(f"{samples} samples is not supported, please choose 1000, 5000, 10000, 20000, or 300000")
    
    params = {'--data-path': dataset.as_named_input('input_files').as_mount(),
              '--shared-model-datastore': datastore.as_mount(),
              '--nodes': nodes, 
              '--procs': processes_per_node, 
              '--samples': samples,
              '--stan-code-file': 'code/model/stan_model.stan'}
    
    mpi_estimator = Estimator(source_directory='..',
                              entry_script='code/mpi_test.py',
                              compute_target=cpu_cluster,
                              node_count=nodes,
                              distributed_training=Mpi(processes_per_node),
                              environment_definition=env,
                              script_params=params,
                              max_run_duration_seconds=3600
                      )
    
    tags = {'nodes': str(nodes), 'processes_per_node': str(processes_per_node), 'samples': str(samples)}
    if test_id:
        tags['test_id'] = str(test_id)
    
    run = stan_exp.submit(mpi_estimator, tags=tags)
    return run

In [69]:
def run_test_suite(nodes=[1, 2, 3, 4], procs=[1, 2, 4, 8, 16], samples=20000):
    """Loop through the passed nodes and procs lists and submit jobs
       Passes a test_id to be used as a tag for experiment runs
    """
    test_id = str(uuid4())
    runs = []
    for node in nodes:
        for proc in procs:
            runs.append(submit_experiment(nodes=node, 
                                          processes_per_node=proc, 
                                          test_id=test_id, 
                                          samples=samples))
    return runs, test_id

In [78]:
# Kick off the test suite
runs = run_test_suite(samples=20000)


In [77]:
from azureml.widgets import RunDetails
RunDetails(runs[0][0]).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [81]:
cores = [1, 2, 3, 4]
procs = [1, 2, 4, 8, 16]

{x * y for x in cores for y in procs}

{1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64}